In [23]:
import os

In [1]:
from code.numbertheory import *

ModuleNotFoundError: No module named 'code.numbertheory'; 'code' is not a package

In [3]:
os.chdir('../code')

In [5]:
from numbertheory import *
from lattices import *
from ecfp import *

In [20]:
def support_full(p):
    a = 0
    cs0 = []
    cs1728 = []
    cond_dic = {}
    while a*a<4*p:
        d,c = discfac(a*a-4*p)
        if d == -3:
            cs0.append(c)
        elif d == -4:
            cs1728.append(c)
        else:
            cond_dic[d] = c
        a+=1
    if len(cs0)>0:
        cond_dic[-3] = 1
        for c in cs0:
            cond_dic[-3] *= c
    if len(cs1728)>0:
        cond_dic[-4] = 1
        for c in cs1728:
            cond_dic[-4] *= c
    return cond_dic

In [22]:
def no_edges_j0(p,l):
    trs = traces_j0(p)
    discs_j0p = [t*t - 4*p for t in trs]
    conds_j0p = [discfac(d)[1] for d in discs_j0p]
    qfs = bqf_iso_verts((1,1,1),l)
    qfs_notendo = [qf for qf in qfs if qf != (1,1,1)]
    no_endos = len(qfs)-len(qfs_notendo)
    if 0 in [c%l for c in conds_j0p]:
        return no_endos+len(qfs_notendo)//3
    else:
        return no_endos

def no_edges_j1728(p,l):
    trs = traces_j1728(p)
    discs_j1728p = [t*t - 4*p for t in trs]
    conds_j1728p = [discfac(d)[1] for d in discs_j1728p]
    qfs = bqf_iso_verts((1,0,1),l)
    qfs_notendo = [qf for qf in qfs if qf != (1,0,1)]
    no_endos = len(qfs)-len(qfs_notendo)
    if 0 in [c%l for c in conds_j1728p]:
        return no_endos+len(qfs_notendo)//2
    else:
        return no_endos

In [21]:
bqf_iso_verts((1,1,1),7)

[(1, 1, 37),
 (3, 3, 13),
 (1, 1, 1),
 (3, 3, 13),
 (1, 1, 37),
 (1, 1, 37),
 (3, 3, 13),
 (1, 1, 1)]

In [ ]:
def divisors(n):
    return pf_to_divisors(primefact(n))

def twisted_phi_sum(d:int,m:int)->int:
    divsm = divisors(m)
    s = 0
    if d == -3:
    return sum([twisted_phi(d,m) for m in divsm])

In [47]:
[twisted_phi_sum(-3,m) for m in range(2,20)]

[2, 2, 4, 3, 6, 3, 8, 5, 10, 5, 14, 5, 10, 10, 16, 7, 18, 7]

In [91]:
def supsingtrace_test(p,l):
    trace = 0
    d3seen = False
    d4seen = False
    a = 0
    while a*a < 4*l:
        d,c = discfac(a*a-4*l)
        qr = quad_rec(d,p)
        if qr < 1:
            while c % p == 0:
                c = c // p
            d0 = d*c*c
            h = len(get_cl_reps(d0))
            if d % p == 0 or d % l == 0:
                if d == -3: 
                    if not d3seen:
                        trace += h
                        d3seen = True
                    else:
                        trace+=h-1
                elif d == -4:
                    if not d4seen:
                        trace += h
                        d4seen = True
                    else:
                        trace+=h-1
                else:  
                    trace += h
            else:
                if d == -3: 
                    if not d3seen:
                        trace += 2*h
                        d3seen = True
                    else:
                        trace+=2*(h-1)
                elif d == -4:
                    if not d4seen:
                        trace += 2*h
                        d4seen = True
                    else:
                        trace+=2*(h-1)
                else:  
                    trace += 2*h
        a+=1                
    return trace

In [132]:
supsingtrace_test(11,13)

18

In [65]:
def support(p):
    pairs = []
    a = 0
    while a*a<4*p:
        d,c = discfac(a*a-4*p)
        pairs.append((d,c))
        a+=1
    return pairs

def support_check(p):
    s = 0
    if p % 3 == 1:
        s-=4
    if p % 4 == 1:
        s-=2
    for pair in support(p):
        d,c = pair
        h = len(get_cl_reps(d*c*c))
        if d % p == 0:
            s += h
        else:
            s += 2*h
    return s

In [84]:
def supsingtrace(p,l):
    trace = 0
    d3seen = False
    d4seen = False
    for pair in support(l):
        d,c = pair
        qr = quad_rec(d,p)
        if qr < 1:
            while c % p == 0:
                c = c // p
            d0 = d*c*c
            h = len(get_cl_reps(d0))
            if d % p == 0 or d % l == 0:
                if d == -3: 
                    if not d3seen:
                        trace += h
                        d3seen = True
                    else:
                        trace+=h-1
                elif d == -4:
                    if not d4seen:
                        trace += h
                        d4seen = True
                    else:
                        trace+=h-1
                else:  
                    trace += h
            else:
                if d == -3: 
                    if not d3seen:
                        trace += 2*h
                        d3seen = True
                    else:
                        trace+=2*(h-1)
                elif d == -4:
                    if not d4seen:
                        trace += 2*h
                        d4seen = True
                    else:
                        trace+=2*(h-1)
                else:  
                    trace += 2*h                
    return trace

In [18]:
def trace_frob(fg:tuple[int,int],p:int)->int:
    f,g = fg
    return - sum([quad_rec(x**3+f*x+g,p) for x in range(p)])

def traces_j0(p):
    if p % 3 == 2:
        return [0]
    else:
        tr1 = trace_frob((0,1),p)
        dc = discfac((tr1**2-4*p))
        b = dc[1]//2
        a = tr1//2
        return [abs(tr1),abs(a)+3*(abs(b)),abs(a)-3*(abs(b))]

def traces_j1728(p):
    if p % 4 == 3:
        return [0]
    else:
        tr1 = trace_frob((1,0),p)
        b = discfac((tr1**2-4*p))[1]
        return [abs(tr1)//2,b]
    
def j2traces(p):
    tracedict = {0:traces_j0(p),(1728%p):traces_j1728(p)}
    for j in range(1,p):
        if j!= (1728%p):
            f = -3*j*(j-1728)//(4*(j**2))
            g = -2*j*(j-1728)**2
            tracedict[j] = abs(trace_frob((f,g),p))
    return tracedict

            

{0: [0], 3: [1, 2], 1: 0, 2: 1, 4: 1}

In [7]:
discfac(-24)

(-24, 1)

In [8]:
discfac(-18)

(-18, 1)

In [9]:
discfac(-12)

(-3, 2)

In [144]:
def x0l_fp_card2(p,l):
    card = 0
    a = 0
    d4seen = 0
    d3seen = 0
    while a*a < 4*p:
        d = a*a - 4*p
        d0,c = discfac(d)
        if c % l == 0:
            card1 = len(get_cl_reps(d))
            card2 = len(get_cl_reps(d//(l**2)))
            if d0 == -3:
                d3seen+=1
                if l % 3 == 1:
                    noj0 = 2+(l-1)//3
                else:
                    noj0 = (l+1)//3
                card+=(card1-1)+l*(card2-1)+noj0
            elif d0 == -4:
                d4seen+=1
                if l % 4 == 1:
                    noj1728 = 2+(l-1)//4
                else:
                    noj1728 = (l+1)//4
                card+=(card1-1)+l*(card2-1)+noj1728
            else:
                card+=(card1)+l*(card2)
        else:
            qr = quad_rec(d0,l)
            if qr >=0:
                h = len(get_cl_reps(d))
                card+=(1+qr)*h
                if d0 == -4:
                    d4seen += (1+qr)*h
                if d0 == -3:
                    d3seen += (1+qr)*h
        a+=1
    if d4seen > 0:
        card-=(d4seen-1)
    if d3seen > 0:
        card-=(d3seen-1)
    return card

In [145]:
x0l_fp_card2(11,5)

14

In [146]:
primes = primesBetween(4,100)

In [147]:
[trace_frob((0,1),p) for p in primes if p %3 == 1]

[-4, 2, 8, -4, -10, 8, 14, -16, -10, -4, 14]

In [149]:
[discfac(trace_frob((0,1),p)**2-4*p) for p in primes if p %3 == 1]

[(-3, 2),
 (-3, 4),
 (-3, 2),
 (-3, 6),
 (-3, 4),
 (-3, 6),
 (-3, 4),
 (-3, 2),
 (-3, 8),
 (-3, 10),
 (-3, 8)]

In [235]:
def x0l_fp_card(p,l):
    card = 0
    if quad_rec(-p,l)==1:
        card += (len(get_cl_reps(-4*p)))
    a = 1
    cond0 = []
    cond1728 = []
    while a*a < 4*p:
        d = a*a-4*p
        d0,c = discfac(d)
        if c % l == 0:
            d1 = d0*((c//l)**2)
            card1 = len(get_cl_reps(d))
            card2 = len(get_cl_reps(d1))
            if d0 == -3:
                card+=(card1-1)+l*(card2-1)
                cond0.append(c)
            elif d0 == -4:
                card+=(card1-1)+l*(card2-1)
                cond1728.append(c)
            else:
                card+=card1+l*card2
        else:
            qr = quad_rec(d0,l)
            if qr >=0:
                card+=(1+qr)*len(get_cl_reps(d))
        a+=1
    # We ignored ordinary curves j = 0, 1728
    if len(cond0) > 0:
        card+=1+quad_rec(-3,l)
        if min([c% l for c in cond0]) == 0:
            if l % 3 == 1:
                card += (l-1)//3
            else:
                card += (l+1)//3
        else:
            card+=1+quad_rec(-3,l)
    if len(cond1728) > 0:
        card+=1+quad_rec(-4,l)
        if min([c% l for c in cond1728]) == 0:
            if l % 4 == 1:
                card += (l-1)//2
            else:
                card += (l+1)//2
    return card

In [14]:
primes = primesBetween(15,400)
errors = {l:[p+1-x0l_fp_card(p,l) for p in primes] for l in [5,7,13]}


In [17]:
errors[13]

[0,
 -2,
 2,
 0,
 -2,
 -4,
 0,
 -2,
 2,
 0,
 2,
 -4,
 -2,
 2,
 -4,
 -2,
 2,
 0,
 -4,
 0,
 -2,
 2,
 -4,
 0,
 -2,
 2,
 0,
 -2,
 0,
 -2,
 -4,
 -2,
 2,
 0,
 2,
 -4,
 2,
 -4,
 0,
 -2,
 -2,
 -2,
 2,
 -4,
 0,
 2,
 -4,
 2,
 0,
 2,
 0,
 -2,
 -4,
 0,
 -2,
 0,
 -2,
 2,
 -4,
 0,
 -2,
 -4,
 2,
 -4,
 0,
 2,
 -2,
 -4,
 -2,
 2,
 0,
 -4]

In [10]:
def discriminants_upto(m:int)->list[int]:
    discs = []
    for d in range(-3,-m,-1):
        if d % 4 == 0 or d % 4 == 1:
            if discfac(d)[1]==1:
                discs.append(d)
    return discs

def disc_qr_vec(d:int,primes:list[int])->dict[int,dict[int,int]]:
    qrvec = {}
    for p in primes:
        qrvec[p] = quad_rec(d,p)
    return qrvec

In [12]:
dlist = discriminants_upto(100)

ds5 = [d for d in dlist if disc_qr_vec(d,primes) ==errors[5]]

In [13]:
ds5

[]

In [233]:
def x0l_fp_card_test(p,l):
    card = {0:0}
    if quad_rec(-p,l)==1:
        card[0] += (len(get_cl_reps(-4*p)))
    cond0 = []
    cond1728 = []
    a = 1
    while a*a < 4*p:
        d = a*a-4*p
        d0,c = discfac(d)
        if c % l == 0:
            d1 = d0*((c//l)**2)
            card1 = len(get_cl_reps(d))
            card2 = len(get_cl_reps(d1))
            if d0 == -3:
                card[a]=(card1-1)+l*(card2-1)
                cond0.append(c)
            elif d0 == -4:
                card[a]=(card1-1)+l*(card2-1)
                cond1728.append(c)
            else:
                card[a]=card1+l*card2
        else:
            qr = quad_rec(d0,l)
            if qr >=0:
                card[a]=(1+qr)*len(get_cl_reps(d))
        a+=1
    # We ignored ordinary curves j = 0, 1728
    if len(cond0) > 0:
        card[-3]=1+quad_rec(-3,l)
        if min([c% l for c in cond0]) == 0:
            if l % 3 == 1:
                card[-3] += (l-1)//3
            else:
                card[-3] += (l+1)//3
    if len(cond1728) > 0:
        card[-4]=1+quad_rec(-4,l)
        if min([c% l for c in cond1728]) == 0:
            if l % 4 == 1:
                card[-4] += (l-1)//2
            else:
                card[-4] += (l+1)//2
    return card

In [234]:
x0l_fp_card_test(11,5)

{0: 4, 2: 2, 3: 2, 5: 2}